[![Abrir en Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/uxue-sudupe/API-adibideak/blob/main/code_examples/es/API_INE_es.ipynb)

# Datos abiertos de INE - Servicios web

## 1. INTRODUCCIÓN

Los datos que se encuentran en la base de datos del INE (INEbase) se pueden obtener a través de servicios web, además de consultarlos en su [página web](https://www.ine.es/dyngs/INEbase/listaoperaciones.htm).

* respuestas en castellano e inglés
* es gratuito y no requiere clave API
* en la reutilización de datos es necesario citar la fuente

La información a la que se podrá acceder a través del servicio API del INE proviene de tres fuentes:

* Base de datos Tempus3
* Repositorio de archivos PC-Axis
* Repositorio de archivos tpx

Utilizaremos la biblioteca [ineapir](https://github.com/es-ine/ineapir) preparada por el INE. Se trata de una biblioteca en lenguaje de programación R que permite buscar y acceder a los datos del INEbase utilizando diferentes filtros.

In [ ]:
# Instalar la biblioteca necesaria (desde Github)

remotes::install_github("es-ine/ineapir")

library(ineapir)

Skipping install of 'ineapir' from a github remote, the SHA1 (6676cb69) has not changed since last install.
  Use `force = TRUE` to force installation



## 2. CONJUNTOS DE DATOS Y CÓDIGOS DE IDENTIFICACIÓN DE INEBASE

No existe un listado o catálogo de todos los conjuntos/tablas de datos que se encuentran en el INEbase.

Sin embargo, en la base de datos Tempus3 se están unificando las operaciones del INE. De momento recoge información de todas las operaciones estadísticas coyunturales y de algunas operaciones estadísticas estructurales. A medida que se integra en la base de datos, cambia el listado de operaciones de Tempus3.

La lista de operaciones disponibles se puede consultar con la función `get_metadata_operations`.


In [ ]:
tempus3_operaciones <- get_metadata_operations()
tempus3_operaciones

Id,Cod_IOE,Nombre,Codigo,Url
<int>,<chr>,<chr>,<chr>,<chr>
4,30147,Estadística de Efectos de Comercio Impagados,EI,NA
6,30211,Índice de Coste Laboral Armonizado,ICLA,NA
7,30168,Estadística de Transmisión de Derechos de la Propiedad,ETDP,NA
10,30256,Indicadores Urbanos,UA,https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176957&idp=1254735976608
13,30219,Estadística del Procedimiento Concursal,EPC,NA
14,30182,Índices de Precios del Sector Servicios,IPS,NA
15,30457,Índice de Precios de la Vivienda (IPV),IPV,NA
16,,Distribución de Nombres,TNOM,https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736177009&idp=1254735572981
18,30180,Índice de Precios de Consumo Armonizado (IPCA),IPCA,NA


In [ ]:
# Si queremos guardar una copia de las operaciones de Tempus3 en formato excel

install.packages("openxlsx")
library(openxlsx)

write.xlsx(tempus3_operaciones, file="Tempus3_Operaciones.xlsx")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



Como a la hora de descargar los datos por API es necesario el número de identificación de tablas o series, aquí se explica cómo localizar el "id".

### 2.1 Tablas/series de la base de datos Tempus3

En este caso, los números de identificación de tablas y series se pueden encontrar sin salir de la interfaz de R. Una vez conocido el código de la operación (extraído del punto superior), se pueden consultar las tablas que se incluyen dentro de dicha operación utilizando la función `get_metadata_tables_operation`.

Para saber el `id` de las series debemos utilizar otra función, la de `get_metadata_series_operation`.

El orden es: Operación > Tabla > Serie > Datos

In [ ]:
# Tablas del Índice de Precios de Consumo (IPC)

IPC_tablas <- get_metadata_tables_operation(operation = "IPC", tip="A")       # En el parámetro "operation" se debe especificar el código de la operación
                                                                              # tip ="A" para literales, de lo contrario se extraen códigos
head(IPC_tablas)

,Id,Nombre,Codigo,T3_Periodicidad,T3_Publicacion,T3_Periodo_ini,Anyo_Periodo_ini,FechaRef_fin,Ultima_Modificacion
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,24077,Índice general nacional. Series desde enero de 1961,NAC,Mensual,Índice de Precios de Consumo,ene.,1961,null,2025-09-12T09:00:00.000+02:00
2,25331,Ponderaciones: general y de grupos ECOICOP,NAC-CCAA-PROV_P,Anual,Índice de Precios de Consumo. Medias anuales,A,2017,null,2024-01-12T09:00:00.000+01:00
3,35083,Índices nacionales: Componentes para el análisis de la COVID-19,2016_NAC,Mensual,Índice de Precios de Consumo,ene.,2017,null,2022-01-14T09:00:00.000+01:00
4,50902,Índices nacionales: general y de grupos ECOICOP,2021_NAC,Mensual,Índice de Precios de Consumo,ene.,2002,null,2025-09-29T09:00:00.000+02:00
5,50908,Índices nacionales a impuestos constantes: general y de grupos ECOICOP,2021_NAC_ICTES,Mensual,Índice de Precios de Consumo,ene.,2011,null,2025-09-12T09:00:00.000+02:00
6,50911,Tasa de variacion del índice general nacional. Series desde enero de 1961,2021_NAC,Mensual,Índice de Precios de Consumo,ene.,1961,null,2025-09-29T09:00:00.000+02:00


In [ ]:
# Series del Índice de Precios de Consumo (IPC)

IPC_series <- get_metadata_series_operation(operation = "IPC", tip ="A")       # En el parámetro "operation" se debe especificar el código de la operación
head(IPC_series)

,COD,T3_Operacion,Nombre,Decimales,T3_Periodicidad,T3_Publicacion,T3_Clasificacion,T3_Escala,T3_Unidad
,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,IPC53262,Índice de Precios de Consumo (IPC),Baleares. Tasa de variación en lo que va de año medicina,1,Mensual,Índice de Precios de Consumo,Base 2001,,Tasas
2,IPC53306,Índice de Precios de Consumo (IPC),Baleares. Dato base. Transporte,3,Mensual,Índice de Precios de Consumo,Base 2001,,Índice
3,IPC53307,Índice de Precios de Consumo (IPC),Baleares. Tasa de variación mensual. Transporte,1,Mensual,Índice de Precios de Consumo,Base 2001,,Tasas
4,IPC53308,Índice de Precios de Consumo (IPC),Baleares. Tasa de variación anual. Transporte,1,Mensual,Índice de Precios de Consumo,Base 2001,,Tasas
5,IPC53311,Índice de Precios de Consumo (IPC),Baleares. Tasa de variación en lo que va de año. Transporte,1,Mensual,Índice de Precios de Consumo,Base 2001,,Tasas
6,IPC53327,Índice de Precios de Consumo (IPC),Baleares. Dato base. Comunicaciones,3,Mensual,Índice de Precios de Consumo,Base 2001,,Índice


In [ ]:
# Si queremos guardar en excel las series del IPC

write.xlsx(IPC_series, file="IPC_series.xlsx")

En algunos casos puede ser más fácil salir de la interfaz R y buscar el código de identificación en la página web. En la página del INE, el código de tablas y series existentes en Tempus3 se encuentra de la siguiente manera:

***Código de identificación de una tabla en Tempus3***

![](https://raw.githubusercontent.com/uxue-sudupe/API-adibideak/main/img/Id_tabla_Tempus.png)



***Código de identificación de una serie en tempus3***

![](https://raw.githubusercontent.com/uxue-sudupe/API-adibideak/main/img/Id_serie_Tempus.png)

### 2.2 Tablas en formato  pc-axis

Hay que salir del interfaz R y buscar el código de identificación en la página web:

***Código de identificación de la tabla pc-axis***

![](https://raw.githubusercontent.com/uxue-sudupe/API-adibideak/main/img/Id_tabla_pc-axis.png)

### 2.3 Tablas en formato tpx

Hay que salir del interfaz R y buscar el código de identificación en la página web:

***Código de identificación de la tabla tpx***

![](https://raw.githubusercontent.com/uxue-sudupe/API-adibideak/main/img/Id_tabla_tpx.png)

## 3. METADATOS ESTRUCTURALES

Los metadatos del servicio web del INE no siguen estándares. Para encontrar los códigos y descripciones de las variables y valores utilizados en las tablas utilizaremos las siguientes funciones:


### 3.1 Variables

La función `get_metadata_variables` recibe los códigos y descripciones de todas las variables del INEbase. También se pueden solicitar variables de una sola operación.

In [ ]:
#  Listado de variables de toda la base de datos INEbase (códigos y descripciones)

variables <- get_metadata_variables()
head(variables)

,Id,Nombre,Codigo
,<int>,<chr>,<chr>
1,349,Totales Territoriales,NAC
2,954,Total,
3,70,Comunidades y Ciudades Autónomas,CCAA
4,516,Nacionalidad,1
5,955,"Cultivos, pastos y huertos",
6,956,SAU y Otras tierras,


In [ ]:
# Si queremos guardar la respuesta en un archivo excel

write.xlsx(variables, "API_INE_variables.xlsx")

In [ ]:
# Listado de variables de una operación

IPC_variables <- get_metadata_variables(operation="IPC")
head(IPC_variables)

,Id,Nombre,Codigo
,<int>,<chr>,<chr>
1,3,Tipo de dato,
2,70,Comunidades y Ciudades Autónomas,CCAA
3,115,Provincias,PROV
4,269,Grupos especiales 2001,
5,270,Rúbricas 2001,
6,349,Totales Territoriales,NAC


### 3.2 Valores

 La función `get_metadata_values`, traduce los códigos y descripciones de los valores que toman las variables del INEbase.

In [ ]:
# "Sexua" aldagaiaren baloreak

variables_18 <- get_metadata_values(variable=18)
variables_18


variables_548 <- get_metadata_values(variable=548)
variables_548

Id,Fk_Variable,Nombre,Codigo,FK_JerarquiaPadres
<int>,<int>,<chr>,<chr>,<list>
451,18,Total,,NULL
452,18,Hombres,1,NULL
453,18,Mujeres,6,NULL
454,18,Ambos sexos,,451
438439,18,No consta,,NULL


Id,Fk_Variable,Nombre,Codigo,FK_JerarquiaPadres
<int>,<int>,<chr>,<chr>,<list>
451,548,Total,,NULL
452,548,Hombres,1,NULL
453,548,Mujeres,6,NULL
454,548,Ambos sexos,,451
438439,548,No consta,,NULL


### 3.3 Variables y valores

Con la función: `get_metadata_series_varval`, se obtiene la correspondencia de las variables y de todos los valores que éstas toman. Es obligatorio introducir el código de una operación.


In [ ]:
varbal<- get_metadata_series_varval(operation="MYH")
varbal


Id,Fk_Variable,Nombre,Codigo
<int>,<int>,<chr>,<chr>
72,3,Dato base,
77,3,Porcentaje,
11406,3,Valor absoluto,
23184,3,Proyección a largo plazo,
23238,3,Por mil,
283908,3,Tasa de actividad,
283909,3,Tasa de empleo de la población,
283910,3,Tasa de paro de la población,
283950,3,Número medio de horas efectivas trabajadas,


## 4. BUSCAR Y OBTENER DATOS

Se pueden descargar todos los datos de una tabla, serie u operación. Se utilizan diferentes funciones en función del elemento que se quiere descargar (tabla, serie u operación).

En caso de querer filtrar los datos antes de la descarga, la sintaxis de los filtros también varía según la fuente de datos (Tempus3, pc-axis o tpx).

### 4.1 Obtener los datos de una serie

Para obtener todos los datos de una serie, se necesita el parámetro `codSerie`, es decir, el código de identificación de la serie. Se utiliza la función `get_data_series`.


In [ ]:
# IPC de los alimentos de Álava (serie IPC271788)

#  Se trata de los datos de esta URL: https://servicios.ine.es/wstempus/js/ES/DATOS_SERIE/IPC271788?date=19900101:20300101

IPC_alimentos_Alava <- get_data_series(codSeries = "IPC271788",
                            tip="A",                     # tip = "A" para literales, en caso contrario aparecerá el código
                            unnest=TRUE,                 # unnest = TRUE para mostrar los datos en una tabla
                            dateStart = "1990/01/01")    # el filtro del tiempo se puede poner de muchas maneras: nlast, dateStart, dateEnd. Si no se indica nada, sólo da el último dato

head(IPC_alimentos_Alava)



,COD,Nombre,T3_Unidad,T3_Escala,Fecha,T3_TipoDato,T3_Periodo,Anyo,Valor
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>
1,IPC271788,Araba/Álava. Alimentos. Índice.,Índice,,2002-01-01T00:00:00.000+01:00,Definitivo,M01,2002,59.637
1.1,IPC271788,Araba/Álava. Alimentos. Índice.,Índice,,2002-02-01T00:00:00.000+01:00,Definitivo,M02,2002,59.291
1.2,IPC271788,Araba/Álava. Alimentos. Índice.,Índice,,2002-03-01T00:00:00.000+01:00,Definitivo,M03,2002,59.355
1.3,IPC271788,Araba/Álava. Alimentos. Índice.,Índice,,2002-04-01T00:00:00.000+02:00,Definitivo,M04,2002,59.765
1.4,IPC271788,Araba/Álava. Alimentos. Índice.,Índice,,2002-05-01T00:00:00.000+02:00,Definitivo,M05,2002,59.983
1.5,IPC271788,Araba/Álava. Alimentos. Índice.,Índice,,2002-06-01T00:00:00.000+02:00,Definitivo,M06,2002,60.071


### 4.1 Obtener la tabla completa

Para obtener todos los datos de una tabla, se necesita el parámetro de `idTable`, es decir, el código de identificación de la tabla. Se utiliza la función `get_data_table`.

In [ ]:
# Número de médicos, en el último año: https://www.ine.es/jaxi/Datos.htm?tpx=75015#_tabs-tabla, id= 75015

medicos <- get_data_table(idTable = 75015)
head(medicos)

,Nombre,Data
,<chr>,<list>
1,"TOTAL, Total, Total","310558, 0"
2,"TOTAL, Total, Hombres","141458, 0"
3,"TOTAL, Total, Mujeres","169100, 0"
4,"TOTAL, Colegiados no jubilados, Total","250754, 0"
5,"TOTAL, Colegiados no jubilados, Hombres","101563, 0"
6,"TOTAL, Colegiados no jubilados, Mujeres","149191, 0"


### 4.2 Filtrar datos

Para filtrar los datos utilizaremos también la función `get_data_table` y el parámetro `filter`. Los filtros se pueden colocar para todas o algunas de las dimensiones/variables y seleccionar una o varias valores/categorías. Será necesario conocer el código de las variables y valores que hemos visto en el punto 3.

Distinguiremos tres tipos posibles de fuentes de tablas (tablas Tempus 3, tablas pc-axis y tablas tpx).

#### 4.2.1 Filtrar datos en Tempus3

Para poder filtrar es necesario conocer los códigos de identificación numérica `Id` de las variables y las categorías que queremos filtrar.

Debemos obtener información de variables y valores de la tabla de interés, por ejemplo [Índices por comunidades autónomas: general y de grupos ECOICOP](https://www.ine.es/jaxiT3/Tabla.htm?t=50913) (Id = 50913).


In [ ]:
metadatos <- get_metadata_table_varval(idTable = 50913)
head(metadatos)

,Id,Fk_Variable,Nombre,Codigo
,<int>,<int>,<chr>,<chr>
1,16473,349,Nacional,00
2,8997,70,Andalucía,01
3,8998,70,Aragón,02
4,8999,70,"Asturias, Principado de",03
5,9000,70,"Balears, Illes",04
6,9001,70,Canarias,05


In [ ]:
# Construimos el filtro y pedimos los datos: tasa de variación interanual del Índice general para la CAE

filtro <- list("3" = "74" ,      # id de #variable = 3, categoría id = 74 tasa de variación interanual
               "762" = "304092", # id de #variable = 762, categoria id = 304092 índice general
               "70" = "9012"     # id = 70, categoría id = 9012 C.A.E
               )

# datos, filtrados
IPC_general_CAE <- get_data_table(idTable = 50913, filter = filtro, unnest = TRUE,
                      tip = "A", nlast = 5)
IPC_general_CAE


,COD,Nombre,T3_Unidad,T3_Escala,Fecha,T3_TipoDato,T3_Periodo,Anyo,Valor
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>
1,IPC255465,País Vasco. Índice general. Variación anual.,Tasas,,2025-08-01T00:00:00.000+02:00,Definitivo,M08,2025,2.8
1.1,IPC255465,País Vasco. Índice general. Variación anual.,Tasas,,2025-07-01T00:00:00.000+02:00,Definitivo,M07,2025,3.0
1.2,IPC255465,País Vasco. Índice general. Variación anual.,Tasas,,2025-06-01T00:00:00.000+02:00,Definitivo,M06,2025,2.8
1.3,IPC255465,País Vasco. Índice general. Variación anual.,Tasas,,2025-05-01T00:00:00.000+02:00,Definitivo,M05,2025,2.4


#### 4.2.2 Filtrar datos en datos pc-axis

Para poder filtrar es necesario conocer los códigos alfanuméricos de identificación `Codigo` de las variables y los valores que queremos filtrar. Por lo tanto, debemos:

Solicitar metadatos de las series que componen la tabla de interés con la función `get_metadata_table_varval`.




In [ ]:
# Población por edad (3 grupos de edad), españoles/extranjeros, sexo y año

metadata <- get_metadata_table_varval(idTable = "t20/e245/p08/l0/01001.px")
metadata

,Nombre,Codigo,Variable.Nombre,Variable.Codigo
,<chr>,<chr>,<chr>,<chr>
1,TOTAL EDADES,totaledades,edad (3 grupos de edad),edad3gruposdeedad
2,TOTAL,total,Españoles/Extranjeros,espanolesextranjeros
3,Ambos sexos,ambossexos,Sexo,sexo
6,Hombres,hombres,Sexo,sexo
9,Mujeres,mujeres,Sexo,sexo
11,Españoles,espanoles,Españoles/Extranjeros,espanolesextranjeros
20,Extranjeros,extranjeros,Españoles/Extranjeros,espanolesextranjeros
29,% Extranjeros,extranjeros~1,Españoles/Extranjeros,espanolesextranjeros
37,0-15 años,015anos,edad (3 grupos de edad),edad3gruposdeedad


In [ ]:

# Hacemos el filtro y pedimos los datos. Por ejemplo, para las mujeres y hombres españoles de todas las edades (mayores de edad = totales) (sexo = mujeres y hombres) (espanolesextranjeras = españolas)

filtro <- list(sexo = c("mujeres", "hombres"),    # si se quiere filtrar más de una categoría,  =c(  , )
               espanolesextranjeros = "espanoles",
               edad3gruposdeedad = "totaledades"
               )

poblacion <- get_data_table (idTable = "t20/e245/p08/l0/01001.px", unnest = TRUE, nlast = 5, filter = filtro)

poblacion


Nombre,NombrePeriodo,Valor,Secreto
<chr>,<chr>,<dbl>,<lgl>
"TOTAL EDADES, Españoles, Hombres",2022,20487167,FALSE
"TOTAL EDADES, Españoles, Hombres",2021,20497801,FALSE
"TOTAL EDADES, Españoles, Hombres",2020,20535713,FALSE
"TOTAL EDADES, Españoles, Hombres",2019,20522283,FALSE
"TOTAL EDADES, Españoles, Hombres",2018,20522724,FALSE
"TOTAL EDADES, Españoles, Mujeres",2022,21445321,FALSE
"TOTAL EDADES, Españoles, Mujeres",2021,21447158,FALSE
"TOTAL EDADES, Españoles, Mujeres",2020,21480929,FALSE
"TOTAL EDADES, Españoles, Mujeres",2019,21467047,FALSE


#### 4.2.3 Filtrar datos en datos tpx

Para poder filtrar es necesario conocer los códigos alfanuméricos de identificación de las variables y las categorías que queremos filtrar. Por lo tanto, debemos hacer esto:


In [ ]:
#  Extracción nacional por tipo de material y año (https://www.ine.es/jaxi/Tabla.htm?tpx=33387&L=0) (Id = 33387)

metadata <- get_metadata_table_varval(idTable = 33387)
head(metadata)

,Nombre,Codigo,Variable.Nombre,Variable.Codigo
,<chr>,<chr>,<chr>,<chr>
1,EXTRACCION NACIONAL,extraccionnacional,tipo de material,tipodematerial
2,1. Biomasa,1biomasa,tipo de material,tipodematerial
3,1.1. Cultivos primarios,11cultivosprimarios,tipo de material,tipodematerial
4,1.1.1. Cereales,111cereales,tipo de material,tipodematerial
5,1.1.2. Raices y tubérculos,112raicesytuberculos,tipo de material,tipodematerial
6,1.1.3. Cultivos azucareros,113cultivosazucareros,tipo de material,tipodematerial


In [ ]:

filtro <- list(tipodematerial = c("1biomasa", "2mineralesmetalicosmineralenbruto",
                                  "3mineralesnometalicos", "4combustiblesfosiles")
               )

materiales <- get_data_table (idTable = 33387, unnest = TRUE, tip = "A", nlast = 1, filter = filtro)
materiales

Nombre,NombrePeriodo,Valor
<chr>,<chr>,<dbl>
1. Biomasa,2023 (avance),107483869
2. Minerales metálicos (mineral en bruto),2023 (avance),23233159
3. Minerales no metálicos,2023 (avance),213294377
4. Combustibles fósiles,2023 (avance),110206


## 5. PARA SABER MÁS

* [Tutorial de ineapir](https://github.com/es-ine/tutorial-ineapir) y [chuleta (cheatsheet)](https://raw.githubusercontent.com/es-ine/ineapir/main/man/figures/ineapir.pdf)

* [Seminario del INE sobre servicios web](https://www.ine.es/ine/eeaapp/4T25_sem06.pdf). 23 de octubre, de 5 horas. El plazo de inscripción está abierto hasta el 15 de octubre.
